# Vietnamese Text Summarization with mT5

Notebook này thực hiện huấn luyện mô hình mT5 cho bài toán tóm tắt văn bản tiếng Việt.

## Hai phương pháp tóm tắt:

### 1. **Extractive Summarization** (Tóm tắt trích xuất)
- Chọn lọc các câu quan trọng có sẵn từ văn bản gốc
- **Ưu điểm**: Ngữ pháp chuẩn, không tạo thông tin sai
- **Nhược điểm**: Tính mạch lạc không cao, có thể rời rạc

### 2. **Abstractive Summarization** (Tóm tắt trừu tượng)
- Sinh câu mới hoàn toàn như con người
- **Ưu điểm**: Tự nhiên hơn, mạch lạc, ngắn gọn
- **Nhược điểm**: Dễ gặp lỗi logic, có thể tạo thông tin sai (hallucination)

---

**Mô hình chính**: mT5 (multilingual T5) - Phù hợp cho tiếng Việt

## 1. Cài đặt môi trường

In [ ]:
print("📁 Mounting Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Drive mounted!")
except:
    print("⚠️  Not running in Colab")

📁 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted!


In [ ]:
# Cài đặt các thư viện cần thiết
!pip install -q transformers datasets accelerate sentencepiece evaluate rouge-score py-rouge
!pip install -q underthesea  # Thư viện NLP tiếng Việt

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate
from tqdm.auto import tqdm
import torch

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cpu


## 2. Chuẩn bị dữ liệu

Sử dụng dataset ViNewsQA hoặc WikiLingua Vietnamese. Nếu bạn có dataset riêng, hãy chỉnh sửa phần này.

In [ ]:
# Cell mới thay thế cell load_data trong notebook

# ============================================================
# 2. Load Data từ CSV (Thay thế cell cũ)
# ============================================================

import pandas as pd
from datasets import Dataset, DatasetDict

data_path = "/content/drive/MyDrive/0000_text_sumarrize"  # Hoặc path tới folder chứa CSV

print("📊 Loading Vietnamese Text Summarization Dataset...")

train_df = pd.read_csv(f"{data_path}/train.csv", names=['document', 'summary', 'keywords'])
val_df = pd.read_csv(f"{data_path}/validation.csv", names=['document', 'summary', 'keywords'])
test_df = pd.read_csv(f"{data_path}/test.csv", names=['document', 'summary', 'keywords'])

# Chỉ giữ document và summary
train_df = train_df[['document', 'summary']]
val_df = val_df[['document', 'summary']]
test_df = test_df[['document', 'summary']]

print(f"✓ Train: {len(train_df):,} samples")
print(f"✓ Validation: {len(val_df):,} samples")
print(f"✓ Test: {len(test_df):,} samples")

# Convert to Dataset
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
})

print(f"\n{dataset}")
print(f"\n📝 Sample:")
print(f"Document: {dataset['train'][0]['document'][:200]}...")
print(f"Summary: {dataset['train'][0]['summary']}")


📊 Loading Vietnamese Text Summarization Dataset...
✓ Train: 15,621 samples
✓ Validation: 1,953 samples
✓ Test: 1,954 samples

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 15621
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 1953
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 1954
    })
})

📝 Sample:
Document: document...
Summary: summary


## 3. Phương pháp 1: Extractive Summarization (Baseline)

Sử dụng TextRank hoặc phương pháp đơn giản dựa trên TF-IDF để chọn câu quan trọng

In [ ]:
# Install nltk
!pip install -q nltk

import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def extractive_summarization(text, num_sentences=3):
    # Dùng NLTK thay vì underthesea
    sentences = nltk_sent_tokenize(text)
    
    if len(sentences) <= num_sentences:
        return text
    
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    sentence_scores = tfidf_matrix.sum(axis=1).A1
    
    top_indices = sentence_scores.argsort()[-num_sentences:][::-1]
    top_indices = sorted(top_indices)
    
    summary = ' '.join([sentences[i] for i in top_indices])
    return summary

# Test
test_text = dataset['train'][0]['document']
print("Extractive summary:")
print(extractive_summarization(test_text, num_sentences=2))

Original text:
document


Extractive summary:
document


## 4. Phương pháp 2: Abstractive Summarization với mT5

Huấn luyện mô hình mT5 để sinh tóm tắt mới hoàn toàn

### 4.1. Chuẩn bị mô hình và tokenizer

In [ ]:
# Load mT5 model - bắt đầu với model nhỏ
model_name = "google/mt5-base"  # Có thể dùng mt5-base hoặc mt5-large nếu có GPU mạnh

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"✓ Loaded model: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pytho

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

✓ Loaded model: google/mt5-base
Model parameters: 582,401,280


### 4.2. Tiền xử lý dữ liệu

In [ ]:
max_input_length = 512
max_target_length = 128
prefix = "summarize: "

def preprocess_function(examples):
    inputs = examples['document']
    targets = examples['summary']

    inputs = [prefix + str(doc) for doc in inputs]
    targets = [str(t) for t in targets]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True,
        padding="max_length"
    )

    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names,
    desc="Tokenizing"
)

print("✓ Preprocessing complete!")
print(tokenized_datasets)

Tokenizing:   0%|          | 0/15621 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/1953 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/1954 [00:00<?, ? examples/s]

✓ Preprocessing complete!
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15621
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1953
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1954
    })
})


### 4.3. Thiết lập metrics đánh giá

In [ ]:
import numpy as np
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """FIXED: Handles OverflowError by clipping token IDs"""
    predictions, labels = eval_pred

    vocab_size = tokenizer.vocab_size

    # FIX: Clip predictions to valid range
    predictions = np.clip(predictions, 0, vocab_size - 1)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 and clip labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = np.clip(labels, 0, vocab_size - 1)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=False
    )

    return {k: round(v * 100, 4) for k, v in result.items()}

print("✓ Metrics function ready!")

✓ Metrics function ready!


### 4.4. Huấn luyện mô hình

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-vietnamese-sum",

    # Evaluation
    eval_strategy="steps",
    eval_steps=500,

    # Training
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,

    # Epochs
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,

    # Generation
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,

    # Performance
    fp16=torch.cuda.is_available(),

    # Logging
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,

    # Best model
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,

    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Trainer initialized!")
total_steps = len(tokenized_datasets['train']) // (4 * 2) * 3
print(f"Total training steps: ~{total_steps:,}")

NameError: name 'tokenizer' is not defined

In [ ]:
print("🚀 Starting training...")
print("This will take ~2-3 hours on T4 GPU")
print("="*50)

trainer.train()

print("\n" + "="*50)
print("✓ Training complete!")

🚀 Starting training...
This will take ~2-3 hours on T4 GPU


NameError: name 'trainer' is not defined

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print("\n" + "="*50)
print("📊 TEST SET RESULTS")
print("="*50)
for key, value in results.items():
    if 'rouge' in key:
        print(f"{key.upper()}: {value:.2f}")

NameError: name 'trainer' is not defined

### 4.5. Đánh giá mô hình

In [ ]:
# Đánh giá trên test set
if isinstance(tokenized_datasets, DatasetDict) and "test" in tokenized_datasets:
    results = trainer.evaluate()
    print("\n" + "="*50)
    print("EVALUATION RESULTS")
    print("="*50)
    for key, value in results.items():
        print(f"{key}: {value:.4f}")
else:
    print("No test set available for evaluation")

## 5. Sử dụng mô hình để tóm tắt văn bản mới

In [ ]:
def generate_summary(text, max_length=128, min_length=30, num_beams=4):
    """
    Sinh tóm tắt abstractive cho văn bản đầu vào
    """
    # Chuẩn bị input
    inputs = tokenizer(
        prefix + text,
        max_length=max_input_length,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    # Generate
    model.to(device)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=num_beams,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

### 5.1. Demo: So sánh Extractive vs Abstractive

In [ ]:
# Văn bản demo
demo_text = """
Việt Nam là một quốc gia nằm ở khu vực Đông Nam Á, có diện tích khoảng 331,000 km² và dân số hơn 98 triệu người.
Việt Nam có nền kinh tế đang phát triển nhanh chóng với tốc độ tăng trưởng GDP hàng năm khoảng 6-7%.
Ngành công nghiệp chế xuất và dịch vụ là hai trụ cột chính của nền kinh tế.
Trong những năm gần đây, Việt Nam đã thu hút được nhiều đầu tư nước ngoài, đặc biệt trong lĩnh vực công nghệ và sản xuất.
Du lịch cũng là một ngành có tiềm năng lớn với nhiều di sản văn hóa thế giới và phong cảnh thiên nhiên tuyệt đẹp.
Chính phủ Việt Nam đang thúc đẩy chuyển đổi số và phát triển kinh tế số để nâng cao năng lực cạnh tranh quốc gia.
""".strip()

print("="*80)
print("VĂN BẢN GỐC")
print("="*80)
print(demo_text)
print("\n" + "="*80)
print("TÓM TẮT EXTRACTIVE (Chọn câu có sẵn)")
print("="*80)
extractive_sum = extractive_summarization(demo_text, num_sentences=2)
print(extractive_sum)
print("\n" + "="*80)
print("TÓM TẮT ABSTRACTIVE (Sinh câu mới với mT5)")
print("="*80)
abstractive_sum = generate_summary(demo_text, max_length=100)
print(abstractive_sum)
print("\n" + "="*80)

### 5.2. Test với văn bản của bạn

In [ ]:
# Nhập văn bản của bạn ở đây
your_text = """
Trí tuệ nhân tạo sinh tạo (Generative AI) đang tạo ra một cuộc cách mạng trong nhiều lĩnh vực.
ChatGPT, DALL-E, và Midjourney là những ví dụ điển hình về sức mạnh của AI sinh tạo.
Các mô hình ngôn ngữ lớn (LLM) như GPT-4 có khả năng hiểu và sinh văn bản tự nhiên ở mức độ ấn tượng.
Trong giáo dục, AI có thể cá nhân hóa trải nghiệm học tập cho từng học sinh.
Trong y tế, AI giúp phát hiện bệnh sớm hơn và đề xuất phương án điều trị hiệu quả.
Tuy nhiên, cũng có những lo ngại về đạo đức, bảo mật, và khả năng AI tạo ra thông tin sai lệch.
Cần có các quy định và khung pháp lý phù hợp để đảm bảo AI phát triển có trách nhiệm.
""".strip()

print("EXTRACTIVE SUMMARY:")
print("-" * 80)
print(extractive_summarization(your_text, num_sentences=2))

print("\n\nABSTRACTIVE SUMMARY (mT5):")
print("-" * 80)
print(generate_summary(your_text))

## 6. Lưu mô hình

In [ ]:
# Lưu mô hình và tokenizer
output_dir = "./mt5-vietnamese-summarization-final"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to {output_dir}")

# Có thể tải lại sau này:
# model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)
# tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [ ]:
# Nén model để download
!zip -r mt5-vietnamese-final.zip mt5-vietnamese-final/

from google.colab import files
files.download('mt5-vietnamese-final.zip')

print("✓ Model ready for download!")

## 7. Kết luận

### So sánh hai phương pháp:

| Tiêu chí | Extractive | Abstractive (mT5) |
|----------|------------|-------------------|
| **Cách hoạt động** | Chọn câu có sẵn | Sinh câu mới |
| **Ngữ pháp** | ✓ Luôn đúng | ~ Có thể có lỗi |
| **Mạch lạc** | ~ Có thể rời rạc | ✓ Tự nhiên hơn |
| **Độ ngắn gọn** | ~ Phụ thuộc câu gốc | ✓ Linh hoạt |
| **Tốc độ** | ✓ Nhanh | ~ Chậm hơn |
| **Tài nguyên** | ✓ Ít | ~ Cần GPU |
| **Thông tin sai** | ✓ Không tạo ra | ✗ Có thể hallucination |

### Khuyến nghị:
- **Extractive**: Dùng khi cần độ chính xác cao, tài nguyên hạn chế
- **Abstractive**: Dùng khi cần tóm tắt tự nhiên, mạch lạc như con người

### Cải thiện thêm:
1. Fine-tune với dataset tiếng Việt lớn hơn
2. Thử mô hình lớn hơn (mt5-base, mt5-large)
3. Kết hợp hai phương pháp: dùng extractive để lọc câu, rồi abstractive để paraphrase
4. Thêm post-processing để kiểm tra logic
5. Sử dụng reinforcement learning để giảm hallucination